# Create YOLOV5 Object Detection Models using MZModel

This example shows you how to use the `dlpy.mzmodel` subpackage to create a YOLOV5 model to perform object detection. The `dlpy.mzmodel` subpackage leverages the [SAS Deep Learning Model Zoo](https://go.documentation.sas.com/doc/en/pgmsascdc/latest/casdlmzpg/titlepage.htm) utilities to manage deep learning models on the CAS server.

## Table of Contents
1. [Set Up Environment](#setup)
2. [Load Data](#prepare)
3. [Build the Model](#build)
4. [Train the Model](#train)
5. [Score the Model and Visualize Scoring Results](#score)

## Set Up Environment <a id="setup"><a>

First, import the various Python and SAS DLPy packages that will be used in this notebook session. Begin by importing the SAS Statistical Wrapper for Analytic Transactions (SWAT). SWAT is the Python interface to SAS CAS. Here is more information about [starting a SAS CAS session with the SWAT package](https://sassoftware.github.io/python-swat/getting-started.html).

In [1]:
import swat as sw
import sys
from dlpy.mzmodel import *
from dlpy.utils import *

Create a CAS session instance. Provide connection information to your running CAS server. For this example, assume the CAS server is hosted at `cloud.example.com` on port `5570`.

In [2]:
# Connect to an existing CAS server
s = sw.CAS('cloud.example.com',5570)

## Load Data <a id="prepare"></a>

In [3]:
s.table.addcaslib(activeonadd=False,
                  datasource={'srctype':'path'},
                  name='dnfs',
                  path='/path/to/data',
                  subdirectories=True)

NOTE: Cloud Analytic Services added the caslib 'dnfs'.


,Name,Type,Description,Path,Definition,Subdirs,Local,Active,Personal,Hidden,Transient,TableRedistUpPolicy
0,dnfs,PATH,,/path/to/data/,,1.0,1.0,0.0,0.0,0.0,0.0,Not Specified


In [4]:
s.setsessopt(caslib='dnfs')

NOTE: 'dnfs' is now the active caslib.


+ Elapsed: 0.000445s, user: 0.000201s, sys: 0.000203s, mem: 0.273mb

In [5]:
s.table.loadTable(
        caslib="dnfs",
        path="coco128/obj_table.txt",
        casout=dict(name="objdet_train128", blocksize="1", replace=True),
    )  

NOTE: Cloud Analytic Services made the file coco128/obj_table.txt available as table OBJDET_TRAIN128 in caslib dnfs.


[caslib]

 'dnfs'

[tableName]

 'OBJDET_TRAIN128'

[casTable]

 CASTable('OBJDET_TRAIN128', caslib='dnfs')

+ Elapsed: 0.0542s, user: 0.0257s, sys: 0.0727s, mem: 82.9mb

The source data that was loaded into CAS table data contains both the image paths and the label paths. These are relative paths based on the cas library.

In [6]:
s.fetch('objdet_train128')

,img_path,label_path
0,coco128/images/train2017/000000000009.bmp,coco128/labels/train2017/000000000009.txt
1,coco128/images/train2017/000000000443.bmp,coco128/labels/train2017/000000000443.txt
2,coco128/images/train2017/000000000025.bmp,coco128/labels/train2017/000000000025.txt
3,coco128/images/train2017/000000000446.bmp,coco128/labels/train2017/000000000446.txt
4,coco128/images/train2017/000000000030.bmp,coco128/labels/train2017/000000000030.txt
5,coco128/images/train2017/000000000450.bmp,coco128/labels/train2017/000000000450.txt
6,coco128/images/train2017/000000000034.bmp,coco128/labels/train2017/000000000034.txt
7,coco128/images/train2017/000000000459.bmp,coco128/labels/train2017/000000000459.txt
8,coco128/images/train2017/000000000036.bmp,coco128/labels/train2017/000000000036.txt
9,coco128/images/train2017/000000000471.bmp,coco128/labels/train2017/000000000471.txt


## Build the Model <a id="build"></a>

Create a YOLOV5 model by using the `MZModel()` constructor. Because `YOLOV5` is available in the Deep Learning Model Zoo, you can specify the architecture by using `type="torchNative"`, `name="yolov5"` and `subtype="small"`. Fit the model to the data set by specifying `num_classes=80` to account for the 80 classes in the data set. If you have a set of weights for the architecture, you can begin training from that set of weights by setting the model_path to the path of the file that contains the weights.

In [7]:
model = MZModel(conn=s, model_type="torchNative", model_name="yolov5", model_subtype="small", num_classes=80, 
                dataset_type= "OBJDETECT", anchors="10 13 16 30 33 23 30 61 62 45 59 119 116 90 156 198 373 326",
                caslib="dnfs",
                model_path="/path/to/model/traced_yolov5s.pt")

The `add_image_transformation()` method enables you to add an image transformation step to resize the image.

In [8]:
model.add_image_transformation(image_size='640', image_resize_type="RETAIN_ASPECTRATIO")

# Train the Model <a id="train"></a>

Use the `train()` method of the `MZModel` class to train the YOLOV5 model. Use `inputs` to specify the column that contains the image paths and `targets` to specify the column that containst the label paths. Pass your optimizer and gpu settings.

In [9]:
optimizer=Optimizer(seed=54321, 
                    algorithm=SGDSolver(lr=0.001, momentum=0.9),
                    batch_size=4,
                    max_epochs=5                   
                    )

In [10]:
model.train(table="objdet_train128", inputs="img_path", targets="label_path", gpu=[0], optimizer=optimizer)

NOTE: 
---------- Starting the dlmzTrain Action ----------
NOTE: No target map from nominal values to numeric values is found.
NOTE: Using this GPU List on your-server.unx.company.com: [0 ].
NOTE: Worker rank 0: your-server.unx.company.com, using up to 1 GPU devices.
NOTE: Using your-server.unx.company.com: GPU 0 processes up to 4 records at a time.
NOTE: Starting dlxexe process on worker your-server.unx.company.com, rank/threadID [0/0], pid= 1486474 device= GPU:0
NOTE: Learning Rate:   0.001 Momentum:     0.9 Weight Decay:       0 Dampening:       0 BatchSize: 4 Rank: 0 Device: 0 Loss: 8.59963
NOTE: The action completed successfully.


[TrainIterHistory]

    Epoch       Loss  Box Loss  Object Loss  Class Loss  Batch Loss       IoU
 0    0.0  15.260728  5.387207     5.871855    4.001666   15.260728  0.691380
 1    1.0  12.065067  4.645655     5.001348    2.418065   12.065067  0.729475
 2    2.0  10.413742  4.262750     4.526867    1.624124   10.413742  0.750554
 3    3.0   9.258885  3.868404     4.132965    1.257515    9.258884  0.770225
 4    4.0   8.599626  3.770609     3.799417    1.029598    8.599626  0.778697

[TuningHistory]

 Empty SASDataFrame
 Columns: [Tuning Iteration, Number of Evaluations, Best Objective]
 Index: []

+ Elapsed: 14.4s, user: 0.0445s, sys: 0.0914s, mem: 90.3mb

## Score the Model and Visualize Scoring Results <a id="score"></a>

Use the `MZModel.score()` method to score the data.

In [11]:
model.score(table="objdet_train128", inputs="img_path", targets="label_path", gpu=[0], batch_size=64)

NOTE: 
---------- Starting the dlmzScore Action ----------
NOTE: Using this GPU List on your-server.unx.company.com: [0 ].
NOTE: No target map from nominal values to numeric values is found.
NOTE: In the score output, expected nominal values will be in raw numeric representation.
NOTE: The specified YAML option string overrides the one saved in the model table.
NOTE: Worker rank 0: your-server.unx.company.com, using up to 1 GPU devices.
NOTE: Using your-server.unx.company.com: GPU 0 processes up to 64 records at a time.
NOTE: Using existing dlxexe process on worker your-server.unx.company.com, rank/threadID [0/0], pid= 1486474 device= GPU:0
NOTE: The action completed successfully.


,Loss,Box Loss,Object Loss,Class Loss,Batch Loss,IoU
0,9.275072,4.315188,3.98774,0.972144,9.275072,0.774544


In the score output table, each row corresponds to an image that can have multiple detected objects. Each detected object in an image spans multiple columns. A more detailed description can be found [here](https://go.documentation.sas.com/doc/en/pgmsascdc/v_052/casactcv/casactcv_image_details18.htm?fromDefault=).

In [12]:
model.table_out.fetch()

,_nObjects_,_Object0_,_P_Object0_,_Object0_x,_Object0_y,_Object0_width,_Object0_height,_Object1_,_P_Object1_,_Object1_x,_Object78_x,_Object78_y,_Object78_width,_Object78_height,_Object79_,_P_Object79_,_Object79_x,_Object79_y,_Object79_width,_Object79_height
0,4.0,50,8.761554e-01,6.498328e-01,7.217432e-01,5.108861e-01,5.258414e-01,50,8.081528e-01,3.053146e-01,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154
1,2.0,65,9.411458e-01,8.915330e-01,5.664850e-01,2.169341e-01,3.191460e-01,0,8.520063e-01,8.798978e-01,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154
2,2.0,23,9.275547e-01,7.706866e-01,4.962384e-01,3.648255e-01,6.622846e-01,23,6.832143e-01,1.886255e-01,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154
3,7.0,58,8.002427e-01,9.431419e-01,3.601360e-01,1.117617e-01,1.568977e-01,0,7.477236e-01,4.609963e-01,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154
4,2.0,75,8.892171e-01,5.046901e-01,5.884227e-01,2.603536e-01,4.651240e-01,58,4.043172e-01,5.220237e-01,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154
5,4.0,40,9.278792e-01,7.989835e-01,7.958960e-02,1.519401e-01,1.423228e-01,40,7.251909e-01,9.370903e-01,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154
6,1.0,22,8.960431e-01,3.537947e-01,4.903257e-01,7.075894e-01,8.587391e-01,,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154
7,2.0,27,9.131442e-01,3.738173e-01,6.370372e-01,1.195215e-01,3.885960e-01,0,7.646556e-01,4.744343e-01,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154
8,2.0,0,8.192990e-01,6.548970e-01,6.320156e-01,6.046782e-01,7.252817e-01,25,7.884791e-01,4.760682e-01,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154
9,2.0,5,9.366448e-01,5.426000e-01,5.070489e-01,7.412134e-01,5.121953e-01,0,4.131285e-01,3.745349e-01,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154,6.013470e-154
